In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib
from sqlalchemy import create_engine, text
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import lxml
import re
import json

In [63]:
# URL новини НБУ
url = 'https://bank.gov.ua/ua/news/all/natsionalniy-bank-u-lyutomu-2025-roku-zastosuvav-do-pyati-bankiv-ta-shesti-nebankivskih-finansovih-ustanov-zahodi-vplivu-za-porushennya-u-sferi-finmonitoringu-ta-valyutnogo-zakonodavstva'

# 1. Завантаження сторінки
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
response.raise_for_status() 

# 2. Парсинг HTML
soup = BeautifulSoup(response.text, "html.parser")

# --- Заголовок ---
title_tag = soup.find("h1", class_="title")
title = title_tag.get_text(strip=True) if title_tag else "—"

# --- Дата ---
pattern = r'"datePublished"\s*:\s*"([^"]+)"'
match = re.search(pattern, response.text)
if match:
    date = match.group(1)
else:
    date = "—"
    
# --- Основний текст ---
content_div = soup.find("div", class_="article-content")
if content_div:
    # дістаємо всі <p> та <li> (деякі абзаци в списках)
    parts = []
    for tag in content_div.find_all(["p", "li"]):
        text = tag.get_text(" ", strip=True)
        if text:
            parts.append(text)
    text = "\n".join(parts)
else:
    text = "—"

# --- Результат ---
print(f"Заголовок: {title}")
print(f"Дата: {date}\n")
print("Текст новини:")
print(text) 

print('----------------------------------')
print(response.text)

Заголовок: Національний банк у лютому 2025 року застосував до п’яти банків та шести небанківських фінансових установ заходи впливу за порушення у сфері фінмоніторингу та валютного законодавства
Дата: 2025-03-07

Текст новини:
Національний банк України за порушення законодавства у сфері запобігання та протидії легалізації (відмиванню) доходів, одержаних злочинним шляхом, фінансуванню тероризму та фінансуванню розповсюдження зброї масового знищення (далі – ПВК/ФТ), валютного законодавства в лютому 2025 року застосував до п’яти банків та шести небанківських фінансових установ заходи впливу.
Заходи впливу застосовано до:
АТ "ОТП БАНК":
1) штраф у розмірі 10 051 000,00 грн за порушення вимог:
пункту 4 частини другої статті 8 Закону України "Про запобігання та протидію легалізації (відмиванню) доходів, одержаних злочинним шляхом, фінансуванню тероризму та фінансуванню розповсюдження зброї масового знищення" (далі – Закон про ПВК/ФТ), що полягає в неналежному виконанні банком обов’язку щодо з

In [54]:
params = "Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=News_NBU;Trusted_Connection=yes;"
quoted = urllib.parse.quote_plus(params)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted}")

conn = engine.raw_connection() 
cursor = conn.cursor()

In [71]:
result = engine.execute("SELECT news_id, link FROM [News_NBU].[dbo].[tb_news_main]")
df_links = pd.DataFrame(result.fetchall())

In [72]:
df_links

,news_id,link
0,1,https://bank.gov.ua/ua/news/all/u-2025-rotsi-u...
1,2,https://bank.gov.ua/ua/news/all/zaprovadjeno-v...
2,3,https://bank.gov.ua/ua/news/all/onovleno-perel...
3,4,https://bank.gov.ua/ua/news/all/biznes-zberig-...
4,5,https://bank.gov.ua/ua/news/all/zatverdjeno-te...
...,...,...
795,796,https://bank.gov.ua/ua/news/all/strahovogo-bro...
796,797,https://bank.gov.ua/ua/news/all/uneseno-zmini-...
797,798,https://bank.gov.ua/ua/news/all/komentar-natsi...
798,799,https://bank.gov.ua/ua/news/all/vistup-golovi-...


In [73]:
for row in df_links.itertuples():
    # Access columns using dot notation
    print(f"id: {row.news_id}, link: {row.link}")

id: 1, link: https://bank.gov.ua/ua/news/all/u-2025-rotsi-uryad-zaluchiv-vid-prodaju---obminu-ovdp-na-auktsionah-ponad-569-mlrd-grn-a-zagalom-uprodovj-voyennogo-stanu--mayje-2-027-mlrd-grn
id: 2, link: https://bank.gov.ua/ua/news/all/zaprovadjeno-vimogi-schodo-oprilyudnennya-bankami-ta-bankivskimi-grupami-prudentsiynoyi-informatsiyi
id: 3, link: https://bank.gov.ua/ua/news/all/onovleno-perelik-inozemnih-valyut-do-yakih-vstanovlyuyetsya-ofitsiyniy-kurs-grivni
id: 4, link: https://bank.gov.ua/ua/news/all/biznes-zberig-strimani-otsinki-svoyeyi-dilovoyi-aktivnosti--pidsumki-opituvannya-pidpriyemstv-u-grudni
id: 5, link: https://bank.gov.ua/ua/news/all/zatverdjeno-tehnichne-zavdannya-dlya-zdiysnennya-otsinki-stiykosti-bankiv-i-bankivskoyi-sistemi-u-2026-rotsi
id: 6, link: https://bank.gov.ua/ua/news/all/natsionalniy-bank-pomyakshuye-nizku-obmejen-dlya-pidtrimannya-oboronnoyi-galuzi
id: 7, link: https://bank.gov.ua/ua/news/all/do-strahovika-zastosovano-zahid-vplivu-u-viglyadi-vidklikannya-li

In [74]:
data = []

for row in df_links.itertuples():
    url = row.link
    news_id = row.news_id

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status() 

    soup = BeautifulSoup(response.text, "html.parser")

    title_tag = soup.find("h1", class_="title")
    title = title_tag.get_text(strip=True) if title_tag else "—"

    pattern = r'"datePublished"\s*:\s*"([^"]+)"'
    match = re.search(pattern, response.text)
    if match:
        date = match.group(1)
    else:
        date = "—"
    
    content_div = soup.find("div", class_="article-content")
    if content_div:
        parts = []
        for tag in content_div.find_all(["p", "li"]):
            text = tag.get_text(" ", strip=True)
            if text:
                parts.append(text)
        text = "\n".join(parts)
    else:
        text = "—"

    data.append({
            'title': title,
            'date': date,
            'text': text,
            'news_id': news_id
        })

In [75]:
df_result = pd.DataFrame(data)

In [76]:
df_result.head()

,title,date,text,news_id
0,У 2025 році уряд залучив від продажу / обміну ...,2026-01-02,"За даними депозитарію НБУ, упродовж 2025 року ...",1
1,Запроваджено вимоги щодо оприлюднення банками ...,2026-01-02,Національний банк України для підвищення еквів...,2
2,"Оновлено перелік іноземних валют, до яких вста...",2026-01-01,Національний банк України з 2 січня 2026 року ...,3
3,Бізнес зберіг стримані оцінки своєї ділової ак...,2026-01-01,Бізнес у грудні зберіг стримані оцінки результ...,4
4,Затверджено технічне завдання для здійснення о...,2025-12-31,Національний банк України затвердив Технічне з...,5


In [77]:
df_result.tail()

,title,date,text,news_id
795,Страхового брокера включено до реєстру,2024-12-12,Національний банк України включив до Державног...,796
796,Унесено зміни до Правил складання та подання з...,2024-12-12,Національний банк вніс зміни до Правил складан...,797
797,Коментар Національного банку щодо рівня інфляц...,2024-12-12,У листопаді 2024 року споживча інфляція в річн...,798
798,Виступ Голови Національного банку Андрія Пишно...,—,"Доброго дня, шановні колеги!\nПравління Націон...",799
799,Національний банк України підвищив облікову ст...,2024-12-12,Правління Національного банку України ухвалило...,800


In [78]:
placeholders = ",".join(["?"] * len(df_result.columns))
sql_query = f"INSERT INTO dbo.tb_news_text (title,date_news,full_text,news_id) VALUES ({placeholders})"

data_to_insert = df_result.values.tolist()

cursor.fast_executemany = True 
cursor.executemany(sql_query, data_to_insert)
conn.commit()
print("Data inserted successfully.")

Data inserted successfully.
